<a href="https://colab.research.google.com/github/aliawofford9317/LSAMP_Python_Course2024/blob/Brendan-Gamor/Brendan17Lesson_4b_Merge_and_Join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def fetch_data():
  import os, shutil
  cwd = os.getcwd()
  if os.path.exists("LSAMP_Python_Course2024"):
    shutil.rmtree("LSAMP_Python_Course2024")
  !git clone https://github.com/aliawofford9317/LSAMP_Python_Course2024.git
  for file in os.listdir("LSAMP_Python_Course2024"):
    if file.endswith((".txt",".csv")):
      shutil.copy("LSAMP_Python_Course2024/{}".format(file),cwd)
fetch_data()

Cloning into 'LSAMP_Python_Course2024'...
remote: Enumerating objects: 210, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 210 (delta 72), reused 1 (delta 1), pack-reused 70
Receiving objects: 100% (210/210), 2.48 MiB | 4.74 MiB/s, done.
Resolving deltas: 100% (108/108), done.


## Merge and Joins
Pandas allows various methods to merge and join dataframes. This allows you to create efficient Dataframes from various sources
- `pd.merge()`
- `pd.join()`
- `pd.concat()`

We will use a display function to correctly display our Dataframes

In [ ]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

Lets combine our dataframes with `pd.merge()`. Our dataframe is automatically joined by the employee column.

In [ ]:
df3 = pd.merge(df1, df2)
df3

### Many to one combination
Here one of the two columns contain duplicate entries

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

### Many to many combination
Here both of the two columns contain duplicate entries

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

Simple merge on employee column. This only works if the `employee` column exists

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

If column names are different on both dataframes, we can use the `left_on` to specify the left column, and `right_on` to specify the other column.

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

The result has a redundant column that we can drop if desired–for example, by using the `drop()` method

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

Sometimes, rather than merging on a column, you would instead like to merge on an index. For example, your data might look like this:

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

You can use the index as the key for merging by specifying the `left_index` and/or `right_index` flags in `pd.merge()`

In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

For convenience, DataFrames implement the `join()` method, which performs a merge that defaults to joining on indices:

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

If you'd like to mix indices and columns, you can combine `left_index` with `right_on` or `left_on` with `right_index` to get the desired behavior

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join. This comes up when a value appears in one key column but not the other.

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

Here we have merged two datasets that have only a single "name" entry in common: Mary. By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join. We can specify this explicitly using the how keyword, which defaults to `"inner"`

In [ ]:
pd.merge(df6, df7, how='inner')

Other options for the how keyword are `'outer', 'left', and 'right'`. An outer join returns a join over the union of the input columns, and fills in all missing values with NAs:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

The left join and right join return joins over the left entries and right entries, respectively. For example:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

The output rows now correspond to the entries in the left input. Using how='right' works in a similar manner.

All of these options can be applied straightforwardly to any of the preceding join types.

### Exercises for participation credit
1. Combine the `student_data1` and `student_data2` into a single dataframe using the `concat()` method. The dataframes must be joined through the rows axis. Both dataframes are provided below.
2. Combine the `student_data3` and `student_data4` into a single dataframe using the common name column. The dataframes must be joined by the name column. You can use `merge()` method.
3. Merge `data1` and `data2` using the keys from the left dataframe only. You will need to use a `left` merge for this.

In [ ]:
# Exercise 1 data
student_data1 = pd.DataFrame({
        'student_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
         'name': ['Danniella Fenton', 'Ryder Storey', 'Bryce Jensen', 'Ed Bernal', 'Kwame Morin'],
        'marks': [200, 210, 190, 222, 199]})

student_data2 = pd.DataFrame({
        'student_id': ['S4', 'S5', 'S6', 'S7', 'S8'],
        'name': ['Scarlette Fisher', 'Carla Williamson', 'Dante Morse', 'Kaiser William', 'Madeeha Preston'],
        'marks': [201, 200, 198, 219, 201]})

In [ ]:
# Exercise 2 data
student_data3 = pd.DataFrame({
        'student_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
         'name': ['Danniella Fenton', 'Ryder Storey', 'Bryce Jensen', 'Ed Bernal', 'Kwame Morin'],
        'marks_test1': [200, 210, 190, 222, 199]})

student_data4 = pd.DataFrame({
        'student_id': ['S1', 'S2', 'S3', 'S4', 'S5'],
        'name': ['Danniella Fenton', 'Ryder Storey', 'Bryce Jensen', 'Ed Bernal', 'Kwame Morin'],
        'marks_test2': [201, 200, 198, 219, 201]})

In [ ]:
# Exercise 3 data
data1 = pd.DataFrame({'key1': ['K0', 'K0', 'K1', 'K2'],
                     'key2': ['K0', 'K1', 'K0', 'K1'],
                     'P': ['P0', 'P1', 'P2', 'P3'],
                     'Q': ['Q0', 'Q1', 'Q2', 'Q3']})
data2 = pd.DataFrame({'key1': ['K0', 'K1', 'K1', 'K2'],
                      'key2': ['K0', 'K0', 'K0', 'K0'],
                      'R': ['R0', 'R1', 'R2', 'R3'],
                      'S': ['S0', 'S1', 'S2', 'S3']})

In [2]:
import pandas as pd

student_data1 = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'age': [20, 21, 19],
    'grade': ['A', 'B', 'C']
})

student_data2 = pd.DataFrame({
    'name': ['David', 'Eva', 'Frank'],
    'age': [22, 23, 21],
    'grade': ['B', 'A', 'B']
})

student_data3 = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'subject': ['Math', 'Science', 'History']
})

student_data4 = pd.DataFrame({
    'name': ['Alice', 'Bob', 'David'],
    'score': [85, 90, 88]
})

data1 = pd.DataFrame({
    'key': ['K0', 'K1', 'K2', 'K3'],
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3']
})

data2 = pd.DataFrame({
    'key': ['K0', 'K1', 'K2'],
    'C': ['C0', 'C1', 'C2'],
    'D': ['D0', 'D1', 'D2']
})

combined_student_data = pd.concat([student_data1, student_data2], axis=0)
print("Combined student_data1 and student_data2:")
print(combined_student_data)

combined_student_data_merged = pd.merge(student_data3, student_data4, on='name')
print("\nCombined student_data3 and student_data4:")
print(combined_student_data_merged)

merged_data = pd.merge(data1, data2, on='key', how='left')
print("\nMerged data1 and data2 using left merge:")
print(merged_data)

Combined student_data1 and student_data2:
      name  age grade
0    Alice   20     A
1      Bob   21     B
2  Charlie   19     C
0    David   22     B
1      Eva   23     A
2    Frank   21     B

Combined student_data3 and student_data4:
    name  subject  score
0  Alice     Math     85
1    Bob  Science     90

Merged data1 and data2 using left merge:
  key   A   B    C    D
0  K0  A0  B0   C0   D0
1  K1  A1  B1   C1   D1
2  K2  A2  B2   C2   D2
3  K3  A3  B3  NaN  NaN
